In [1]:
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

In [2]:
model = pyo.ConcreteModel()
model.x = pyo.Var([1, 2, 3], domain=pyo.NonNegativeReals)
model.objective_o = pyo.Objective(expr=2*model.x[1] + model.x[2] + model.x[3], sense=pyo.maximize)
model.constraint_c1 = pyo.Constraint(expr=2*model.x[1] + 3*model.x[2] - model.x[3] <= 9)
model.constraint_c2 = pyo.Constraint(expr=2*model.x[2] + 3*model.x[3] >= 4)
model.constraint_c3 = pyo.Constraint(expr=model.x[1] + model.x[3] == 6)

In [3]:
opt = SolverFactory('glpk')
results = opt.solve(model)
model.display()

Model unknown

  Variables:
    x : Size=3, Index={1, 2, 3}
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :   4.8 :  None : False : False : NonNegativeReals
          2 :     0 :   0.2 :  None : False : False : NonNegativeReals
          3 :     0 :   1.2 :  None : False : False : NonNegativeReals

  Objectives:
    objective_o : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True :  11.0

  Constraints:
    constraint_c1 : Size=1
        Key  : Lower : Body : Upper
        None :  None :  9.0 :   9.0
    constraint_c2 : Size=1
        Key  : Lower : Body               : Upper
        None :   4.0 : 3.9999999999999996 :  None
    constraint_c3 : Size=1
        Key  : Lower : Body : Upper
        None :   6.0 :  6.0 :   6.0


In [4]:
# Inspect the solver results for multiple optimal solutions and degeneracy
print("=" * 60)
print("ANALYZING SOLVER RESULTS FOR MULTIPLE SOLUTIONS & DEGENERACY")
print("=" * 60)

# 1. Check solver status and termination condition
print("\n1. SOLVER STATUS:")
print(f"   Status: {results.solver.status}")
print(f"   Termination Condition: {results.solver.termination_condition}")

# 2. Solution information
print("\n2. SOLUTION INFORMATION:")
if hasattr(results, 'solver') and hasattr(results.solver, 'statistics'):
    print(f"   Statistics: {results.solver.statistics}")

# 3. Check for infeasibility or unboundedness
print("\n3. SOLUTION FEASIBILITY:")
print(f"   Solution is feasible: {results.solver.termination_condition == pyo.TerminationCondition.optimal}")

# 4. Examine reduced costs (dual variables for non-basic variables)
print("\n4. REDUCED COSTS (Dual values for variables):")
print("   (Zero reduced costs for non-basic variables indicate alternative optimal solutions)")
for i in model.x:
    var_value = model.x[i].value
    # Get dual (reduced cost) - reduced cost is available via the suffix
    print(f"   x[{i}]: value={var_value}")

# Add dual suffix to get reduced costs
from pyomo.environ import Suffix
model.dual = Suffix(direction=Suffix.IMPORT)
model.ipopt_zL_out = Suffix(direction=Suffix.IMPORT)
model.ipopt_zU_out = Suffix(direction=Suffix.IMPORT)

# Solve again to capture dual values
results = opt.solve(model, tee=False)

print("\n5. DUAL VALUES (Shadow prices from constraints):")
for constraint in model.component_data_objects(pyo.Constraint, active=True):
    if constraint in model.dual:
        print(f"   {constraint.name}: dual = {model.dual[constraint]}")

print("\n6. VARIABLE REDUCED COSTS:")
for var in model.component_data_objects(pyo.Var, active=True):
    if var in model.dual:
        print(f"   {var.name}: reduced cost = {model.dual[var]}")
    else:
        # Check if variable is at lower or upper bound
        print(f"   {var.name}: value = {var.value}, lower = {var.lb}, upper = {var.ub}")

print("\n7. DEGENERACY ANALYSIS:")
print("   Note: GLPK simplex method may not directly report degeneracy.")
print("   Indicators of degeneracy:")
print("   - Multiple constraints active (equality or inequality at boundary)")
print("   - Zero reduced costs for non-basic variables")

# Check for constraints at their bounds
active_constraints = []
for constraint in model.component_data_objects(pyo.Constraint, active=True):
    body_value = pyo.value(constraint.body)
    lower = constraint.lower
    upper = constraint.upper
    
    is_active = False
    if lower is not None and abs(body_value - lower) < 1e-6:
        is_active = True
    if upper is not None and abs(body_value - upper) < 1e-6:
        is_active = True
    
    if is_active:
        active_constraints.append((constraint.name, body_value))

print(f"\n   Active constraints (at their bounds): {len(active_constraints)}")
for name, value in active_constraints:
    print(f"     - {name}")

print("\n" + "=" * 60)

ANALYZING SOLVER RESULTS FOR MULTIPLE SOLUTIONS & DEGENERACY

1. SOLVER STATUS:
   Status: ok
   Termination Condition: optimal

2. SOLUTION INFORMATION:
   Statistics: 
Branch and bound: 
  Number of bounded subproblems: 0
  Number of created subproblems: 0


3. SOLUTION FEASIBILITY:
   Solution is feasible: True

4. REDUCED COSTS (Dual values for variables):
   (Zero reduced costs for non-basic variables indicate alternative optimal solutions)
   x[1]: value=4.8
   x[2]: value=0.2
   x[3]: value=1.2

5. DUAL VALUES (Shadow prices from constraints):
   constraint_c1: dual = 0.333333333333333
   constraint_c2: dual = -0.0
   constraint_c3: dual = 1.33333333333333

6. VARIABLE REDUCED COSTS:
   x[1]: value = 4.8, lower = 0, upper = None
   x[2]: value = 0.2, lower = 0, upper = None
   x[3]: value = 1.2, lower = 0, upper = None

7. DEGENERACY ANALYSIS:
   Note: GLPK simplex method may not directly report degeneracy.
   Indicators of degeneracy:
   - Multiple constraints active (equality 

## Question 2

$$
\begin{aligned}
\text{Maximize } z = & \left( 7.9 \times \sum_{j=A}^{C} x_{1j} + 6.9 \times \sum_{j=A}^{C} x_{2j} + 5 \times \sum_{j=A}^{C} x_{3j} \right) \\
& - \left( 0.6 \times \sum_{i=1}^{3} x_{iA} + 0.52 \times \sum_{i=1}^{3} x_{iB} + 0.48 \times \sum_{i=1}^{3} x_{iC} \right)
\end{aligned}
$$

$$
\begin{aligned}
\text{subject to:} \quad & x_{1A} - 0.60 \times \sum_{j=A}^{C} x_{1j} \leq 0 \\
& x_{1C} - 0.20 \times \sum_{j=A}^{C} x_{1j} \geq 0 \\
& x_{2A} - 0.15 \times \sum_{j=A}^{C} x_{2j} \leq 0 \\
& x_{2C} - 0.60 \times \sum_{j=A}^{C} x_{2j} \geq 0 \\
& x_{3C} - 0.50 \times \sum_{j=A}^{C} x_{3j} \geq 0 \\
& \sum_{i=1}^{3} x_{iA} \leq 4000 \\
& \sum_{i=1}^{3} x_{iB} \leq 5000 \\
& \sum_{i=1}^{3} x_{iC} \leq 2500 \\
& 0 \leq x_{ij}, \quad \forall i \in \{1,2,3\}, \quad \forall j \in \{A,B,C\}
\end{aligned}
$$

# Question 2 - Solution

In [5]:
# Question 2: Production Allocation Optimization Problem
# Create a new model for Question 2
model2 = pyo.ConcreteModel()

# Define sets
model2.products = pyo.Set(initialize=[1, 2, 3])  # Products: 1, 2, 3
model2.destinations = pyo.Set(initialize=['A', 'B', 'C'])  # Destinations: A, B, C

# Define decision variables: x[i,j] = quantity of product i sent to destination j
model2.x = pyo.Var(model2.products, model2.destinations, domain=pyo.NonNegativeReals)

# Define parameters for revenue coefficients
revenue_coeff = {1: 7.9, 2: 6.9, 3: 5.0}
cost_coeff = {'A': 0.6, 'B': 0.52, 'C': 0.48}

# Define objective function
def objective_expr(model):
    revenue = sum(revenue_coeff[i] * sum(model.x[i, j] for j in model.destinations) 
                  for i in model.products)
    cost = sum(cost_coeff[j] * sum(model.x[i, j] for i in model.products) 
               for j in model.destinations)
    return revenue - cost

model2.objective = pyo.Objective(expr=objective_expr, sense=pyo.maximize)

# Define constraints
# Constraint 1: Product 1 distribution to A
model2.constraint_1a = pyo.Constraint(
    expr=model2.x[1, 'A'] - 0.60 * sum(model2.x[1, j] for j in model2.destinations) <= 0
)

# Constraint 1b: Product 1 distribution to C (lower bound)
model2.constraint_1c = pyo.Constraint(
    expr=model2.x[1, 'C'] - 0.20 * sum(model2.x[1, j] for j in model2.destinations) >= 0
)

# Constraint 2a: Product 2 distribution to A
model2.constraint_2a = pyo.Constraint(
    expr=model2.x[2, 'A'] - 0.15 * sum(model2.x[2, j] for j in model2.destinations) <= 0
)

# Constraint 2c: Product 2 distribution to C (lower bound)
model2.constraint_2c = pyo.Constraint(
    expr=model2.x[2, 'C'] - 0.60 * sum(model2.x[2, j] for j in model2.destinations) >= 0
)

# Constraint 3c: Product 3 distribution to C (lower bound)
model2.constraint_3c = pyo.Constraint(
    expr=model2.x[3, 'C'] - 0.50 * sum(model2.x[3, j] for j in model2.destinations) >= 0
)

# Capacity constraints for each destination
model2.capacity_A = pyo.Constraint(
    expr=sum(model2.x[i, 'A'] for i in model2.products) <= 4000
)

model2.capacity_B = pyo.Constraint(
    expr=sum(model2.x[i, 'B'] for i in model2.products) <= 5000
)

model2.capacity_C = pyo.Constraint(
    expr=sum(model2.x[i, 'C'] for i in model2.products) <= 2500
)

print("Model 2 created successfully!")
print(f"Number of variables: {len(model2.component_map(pyo.Var))}")
print(f"Number of constraints: {len(model2.component_map(pyo.Constraint))}")

Model 2 created successfully!
Number of variables: 1
Number of constraints: 8


In [6]:
# Solve the model
opt2 = SolverFactory('glpk')

# Add dual suffix to capture shadow prices
model2.dual = Suffix(direction=Suffix.IMPORT)

# Solve
results2 = opt2.solve(model2, tee=False)

print("=" * 70)
print("QUESTION 2 - OPTIMIZATION RESULTS")
print("=" * 70)

# 1. Solver Status
print("\n1. SOLVER STATUS:")
print(f"   Status: {results2.solver.status}")
print(f"   Termination Condition: {results2.solver.termination_condition}")
print(f"   Message: {results2.solver.message if hasattr(results2.solver, 'message') else 'N/A'}")

# 2. Optimal Objective Value
print("\n2. OPTIMAL OBJECTIVE VALUE:")
print(f"   z* = {pyo.value(model2.objective):.2f}")

# 3. Optimal Solution
print("\n3. OPTIMAL SOLUTION (Decision Variables):")
print("   Distribution Matrix x[i,j] (Product i to Destination j):")
print("   " + "-" * 60)
print("   {:15} {:>12} {:>12} {:>12}".format("Product", "Destination A", "Destination B", "Destination C"))
print("   " + "-" * 60)

for i in model2.products:
    values = [pyo.value(model2.x[i, j]) for j in model2.destinations]
    print("   {:15} {:>12.2f} {:>12.2f} {:>12.2f}".format(
        f"Product {i}", values[0], values[1], values[2]))

# Summary statistics
print("\n   Summary by Product:")
for i in model2.products:
    total = sum(pyo.value(model2.x[i, j]) for j in model2.destinations)
    print(f"     Product {i} Total: {total:.2f}")

print("\n   Summary by Destination:")
for j in model2.destinations:
    total = sum(pyo.value(model2.x[i, j]) for i in model2.products)
    print(f"     Destination {j} Total: {total:.2f}")

# 4. Constraint Analysis
print("\n4. CONSTRAINT ANALYSIS:")
print("   " + "-" * 70)

# Product distribution constraints
constraints_info = [
    ('constraint_1a', 'x[1,A] ≤ 0.60·Σx[1,j]', model2.constraint_1a),
    ('constraint_1c', 'x[1,C] ≥ 0.20·Σx[1,j]', model2.constraint_1c),
    ('constraint_2a', 'x[2,A] ≤ 0.15·Σx[2,j]', model2.constraint_2a),
    ('constraint_2c', 'x[2,C] ≥ 0.60·Σx[2,j]', model2.constraint_2c),
    ('constraint_3c', 'x[3,C] ≥ 0.50·Σx[3,j]', model2.constraint_3c),
    ('capacity_A', 'Σx[i,A] ≤ 4000', model2.capacity_A),
    ('capacity_B', 'Σx[i,B] ≤ 5000', model2.capacity_B),
    ('capacity_C', 'Σx[i,C] ≤ 2500', model2.capacity_C),
]

print("   Constraint Status:")
for name, desc, constraint in constraints_info:
    body = pyo.value(constraint.body)
    lower = constraint.lower if constraint.lower is not None else '-∞'
    upper = constraint.upper if constraint.upper is not None else '+∞'
    
    # Determine if binding
    is_binding = False
    if constraint.lower is not None and abs(body - constraint.lower) < 1e-5:
        is_binding = True
    if constraint.upper is not None and abs(body - constraint.upper) < 1e-5:
        is_binding = True
    
    binding_str = "BINDING" if is_binding else "slack"
    
    print(f"   {name:15} | {desc:30} | Body: {body:8.2f} | {binding_str:8}")
    if constraint in model2.dual:
        print(f"                    | Shadow Price (Dual): {model2.dual[constraint]:.6f}")

# 5. Revenue and Cost Breakdown
print("\n5. REVENUE AND COST BREAKDOWN:")
revenue_by_product = {}
for i in model2.products:
    total_produced = sum(pyo.value(model2.x[i, j]) for j in model2.destinations)
    revenue_by_product[i] = revenue_coeff[i] * total_produced
    print(f"   Product {i} Revenue: {revenue_coeff[i]} × {total_produced:.2f} = {revenue_by_product[i]:.2f}")

total_revenue = sum(revenue_by_product.values())
print(f"   Total Revenue: {total_revenue:.2f}")

print("\n   Costs by Destination:")
cost_by_destination = {}
for j in model2.destinations:
    total_to_dest = sum(pyo.value(model2.x[i, j]) for i in model2.products)
    cost_by_destination[j] = cost_coeff[j] * total_to_dest
    print(f"   Destination {j} Cost: {cost_coeff[j]} × {total_to_dest:.2f} = {cost_by_destination[j]:.2f}")

total_cost = sum(cost_by_destination.values())
print(f"   Total Cost: {total_cost:.2f}")

print(f"\n   Net Profit: {total_revenue:.2f} - {total_cost:.2f} = {total_revenue - total_cost:.2f}")

# 6. Dual Values Summary
print("\n6. DUAL VALUES (Shadow Prices) - Sensitivity Analysis:")
print("   " + "-" * 70)
print("   Constraint                      | Dual Value (Shadow Price)")
print("   " + "-" * 70)
for name, desc, constraint in constraints_info:
    if constraint in model2.dual:
        dual_val = model2.dual[constraint]
        print(f"   {name:32} | {dual_val:20.6f}")
    else:
        print(f"   {name:32} | {'N/A':20}")

# 7. Degeneracy and Alternative Solutions Check
print("\n7. DEGENERACY AND ALTERNATIVE SOLUTIONS ANALYSIS:")
binding_constraints = sum(1 for _, _, c in constraints_info 
                         if (c.lower is not None and abs(pyo.value(c.body) - c.lower) < 1e-5) or
                            (c.upper is not None and abs(pyo.value(c.body) - c.upper) < 1e-5))
num_variables = len(list(model2.component_data_objects(pyo.Var, active=True, descend_into=True)))

print(f"   Number of active variables: {num_variables}")
print(f"   Number of binding constraints: {binding_constraints}")
print(f"   Degrees of freedom: {num_variables - binding_constraints}")

if binding_constraints > num_variables:
    print(f"   → DEGENERATE: More binding constraints than variables")
elif binding_constraints == num_variables:
    print(f"   → POSSIBLY DEGENERATE: Equal binding constraints and variables")
else:
    print(f"   → NOT DEGENERATE: Fewer binding constraints than variables")

# Check for zero reduced costs on non-basic variables
zero_duals = sum(1 for _, _, c in constraints_info if constraint in model2.dual and abs(model2.dual[constraint]) < 1e-6)
if zero_duals > 0:
    print(f"   → ALTERNATIVE OPTIMAL SOLUTIONS: {zero_duals} constraints have zero shadow price")
else:
    print(f"   → UNIQUE OPTIMAL SOLUTION: No constraints with zero shadow price")

print("\n" + "=" * 70)

QUESTION 2 - OPTIMIZATION RESULTS

1. SOLVER STATUS:
   Status: ok
   Termination Condition: optimal
   Message: <undefined>

2. OPTIMAL OBJECTIVE VALUE:
   z* = 84650.00

3. OPTIMAL SOLUTION (Decision Variables):
   Distribution Matrix x[i,j] (Product i to Destination j):
   ------------------------------------------------------------
   Product         Destination A Destination B Destination C
   ------------------------------------------------------------
   Product 1            4000.00      5000.00      2500.00
   Product 2               0.00         0.00         0.00
   Product 3               0.00         0.00         0.00

   Summary by Product:
     Product 1 Total: 11500.00
     Product 2 Total: 0.00
     Product 3 Total: 0.00

   Summary by Destination:
     Destination A Total: 4000.00
     Destination B Total: 5000.00
     Destination C Total: 2500.00

4. CONSTRAINT ANALYSIS:
   ----------------------------------------------------------------------
   Constraint Status:
   